In [ ]:
import tensorflow as tf
import numpy      as np

import pathlib
import config
import data
import os

from tensorflow.keras.models import load_model

"""
Documentation:
- numpy
    1. array()
        - https://numpy.org/doc/stable/reference/generated/numpy.array.html?highlight=array#numpy.array
    2. load()
        - https://numpy.org/doc/stable/reference/generated/numpy.load.html?highlight=load#numpy.load
    3. pad()
        - https://numpy.org/doc/stable/reference/generated/numpy.pad.html?highlight=pad#numpy.pad
- os
    1. path.sep
- pathlib
    1. Path(), /, glob(), .name, .stem
        - https://docs.python.org/3/library/pathlib.html
- tensorflow
    - data.Dataset
        1. batch(), from_tensor_slices(), list_files(), map(), prefetch()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/data/Dataset
    - keras
        - applications.inception_v3
            1. preprocess_input()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/applications/inception_v3
        - models
            1. load_model()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/models/load_model
            2. Sequential()
                1. predict(), evaluate()
                    - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/Sequential
        - preprocessing.image
            1. img_to_array(), load_img()
                - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/keras/preprocessing/image
    - image
        1. convert_image_dtype(), decode_jpeg(), resize()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/image
    -io
        1. read_file()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/io/read_file
    - strings
        1. split()
            - https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/strings/split
Sources:
    1. Input pipeline using tf.data
        - https://www.tensorflow.org/tutorials/load_data/images
    2. Loading numpy arrays
        - https://www.tensorflow.org/tutorials/load_data/numpy
        - https://www.tensorflow.org/guide/data#consuming_numpy_arrays
"""


In [ ]:
AUTOTUNE   = tf.data.experimental.AUTOTUNE
dataObj    = data.Data()
numClasses = dataObj.numClasses
classNames = np.array(dataObj.classes)

In [ ]:
"""
Function Name: getBestSavedModel
Number of parameters: 1
List of parameters:
    1. modelCheckpointPath | pathlib.Path | Path to the CNN model checkpoints.
Pre-condition:
    1. modelCheckpointPath exists.
Post-condition:
    1. Returns the path to the best model in the 'modelCheckpointPath' directory. The best model, is the one
       that has the smallest loss.
"""
def getBestSavedModel(modelCheckpointPath):
    dirPaths  = sorted(modelCheckpointPath.glob("*"))
    minLoss   = float('inf')
    bestModel = ""
    for dirPath in dirPaths:
        modelname = pathlib.Path(dirPath.name).stem
        loss = float(modelname.split("_")[-1])
        if loss <= minLoss:
            bestModel = dirPath
            minLoss   = loss
    return str(bestModel)

In [ ]:
"""
Function Name: evaluateCNN
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Returns the results (evaluation) of a CNN model.
"""
def evaluateCNN():
    def getDataSet(dataDirectory):
        imagePathsDataset = tf.data.Dataset.list_files(str(dataDirectory/'*/*'))
        return imagePathsDataset
    
    def getLabeledData(imagePath):
        def getLabel(imagePath):
            # convert the path to a list of path components
            parts = tf.strings.split(imagePath, os.path.sep)
            # The second to last is the class-directory
            return parts[-2] == classNames
    
        def decodeImg(img):
            # convert the compressed string to a 3D uint8 tensor
            img = tf.image.decode_jpeg(img, channels=3)
            # Use `convert_image_dtype` to convert to floats in the [0,1] range.
            img = tf.image.convert_image_dtype(img, tf.float32)
            # resize the image to the desired size.
            return tf.image.resize(img, [299, 299])
    
        label = getLabel(imagePath)
        # load the raw data from the file as a string
        img = tf.io.read_file(imagePath)
        img = decodeImg(img)
        return img, label
    
    BATCH_SIZE = 32
    cf         = config.Config()
    rootPath   = pathlib.Path(cf.rootPath)
    framesPath = pathlib.Path(cf.framesPath)
    testDataDirectory   = framesPath/'Test'
    modelCheckpointPath = rootPath/'Callbacks'/'CNN'/f'{numClasses}'/'ModelCheckpoint'

    testDataset = getDataSet(testDataDirectory)
    testDataset = testDataset.map(getLabeledData, num_parallel_calls = AUTOTUNE)
    testDataset = testDataset.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)

    savedModelPath = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    
    savedModel = load_model(savedModelPath)
    results = savedModel.evaluate(testDataset)
    return results

results = evaluateCNN()
print("Loss: ", results[0], "Accuracy: ", results[1])

In [ ]:
"""
Function Name: evaluateRNN
Number of parameters: 0
List of parameters: n/a
Pre-condition: n/a
Post-condition:
    1. Returns the results (evaluation) of a RNN model.
"""
def evaluateRNN():
    def getDataset(sequences, labels):
        dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
        return dataset
    
    def getSequences(datasetType):
        maxFrameCount = dataObj.getMaxFrameCount()
        sequences = []
        labels    = []
        sequencesPath = pathlib.Path(config.Config().sequencesPath)
        for dataRow in dataObj.data[:]:
            if(datasetType == dataRow[0]):
                sequencePath = sequencesPath/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
                sequence     = np.load(sequencePath)
                sequence     = np.pad(sequence, ((0, maxFrameCount - int(dataRow[3])), (0, 0)), 'edge')
                sequences.append(sequence)
                label = dataObj.getClassIndex(dataRow[1])
                labels.append(label)
        return np.array(sequences), np.array(labels, dtype=np.uint8)
    
    BATCH_SIZE = 32
    cf         = config.Config()
    rootPath   = pathlib.Path(cf.rootPath)
    modelCheckpointPath = rootPath/'Callbacks'/'RNN'/f'{numClasses}'/'ModelCheckpoint'
    
    testSequences, testLabels = getSequences('Test')
    testDataset = getDataset(testSequences, testLabels)
    testDataset = testDataset.batch(BATCH_SIZE).prefetch(buffer_size = AUTOTUNE)
    
    savedModelPath = "" # you can insert the path to a saved model here
    if savedModelPath == "":
        savedModelPath = getBestSavedModel(modelCheckpointPath)
    
    savedModel = load_model(savedModelPath)
    results = savedModel.evaluate(testDataset)
    return results

results = evaluateRNN()
print("Loss: ", results[0], "Accuracy: ", results[1])